In [1]:
import requests
import pandas as pd
import time

import warnings
warnings.filterwarnings('ignore')

In [2]:
api_key = 'Sxwiysbra37brnJK9Fp12Z81vXvU0IEN'

In [3]:
def df_aution_sold(itemName, limit=100):
    url = f'https://api.neople.co.kr/df/auction-sold'
    params = {
        'itemName' : itemName,
        'limit' : limit,
        'apikey' : api_key
    }
    
    return requests.get(url, params = params)

def df_search_item(itemName = '', item_hashtag = ''):
    url = 'https://api.neople.co.kr/df/items'

    params = {
        'limit' : 30,
        'apikey' : api_key,
        'wordType' : 'full'
    }

    if itemName != '':
        params['itemName'] = itemName
    if item_hashtag != '':
        params['hashtag'] = item_hashtag

    return requests.get(url, params=params)

In [4]:
def check_new_items(new_item_dates, itemId, new_check_lst, isFirst):
    
    new_items = []
    
    # 체크를 제일 처음할때 - 계속 켜놓지 못하니까 불러와서 확인하기 
    if isFirst:
        print('Frist Search')
        db_saled_dates = get_auction_saled_record_from_db(itemId)
        
        for new_item_date in new_item_dates:
            if new_item_date not in db_saled_dates :
                new_items.append(new_item_date)
            new_check_lst[itemId].append(new_item_date)   

    #처음 탐색이 아닌경우
    else:
        for new_item_date in new_item_dates:
            if new_item_date not in new_check_lst[itemId] :
                new_items.append(new_item_date)
                new_check_lst[itemId].append(new_item_date)
                
    return new_items

def regi_item(df):
    use_cols = ['soldDate', 'itemId', 'count', 'unitPrice']

    excecuteDB(df[use_cols])
    

# 경매장에서 아이템 검색
def searching_item(target):
    return pd.DataFrame(df_aution_sold(target).json()['rows']) 


def search_and_regi_item(target, new_check_lst, isFrist):
    df = searching_item(target)
    new_item_dates = df['soldDate']
    
    new_item_dates = check_new_items(new_item_dates, item_dicts[target], new_check_lst, isFrist)
    
    df = df[df['soldDate'].isin(new_item_dates)] # 새롭게 등록된 아이템 필터링
    if len(df) > 0:
        print(f'{target} New 판매 아이템 갯수 / {len(new_item_dates)}')    
    
    regi_item(df) # DB등록

In [5]:
search_item = '특별한 황금 주화'

danjin_gold_token = pd.DataFrame(df_search_item(itemName=search_item).json()['rows'])
enter_tickets = danjin_gold_token[danjin_gold_token['itemName'].str.contains('단진')]
enter_tickets = danjin_gold_token[~danjin_gold_token['itemName'].str.contains('황금 주화')]
search_names = danjin_gold_token['itemName'].values
search_ids = danjin_gold_token['itemId'].values

item_dicts = {item_name : item_id for item_name, item_id in zip(search_names, search_ids)}

item_dicts

{'대형 단진의 특별한 황금 주화 상자': 'c4df9933c11b3399b21f362ba4395523',
 '소형 단진의 특별한 황금 주화 상자': '83f74cef66c5fa5c1d30137ed125941c'}

In [12]:
new_check_lst = {search_id : [] for search_id in search_ids}
isFrist = True

while True:
    for target in search_names:
        try:
            search_and_regi_item(target, new_check_lst, isFrist)
        except Exception as e:
            print(e)
            time.sleep(2)
            
    isFrist = False
    time.sleep(1)

Frist Search
대형 단진의 특별한 황금 주화 상자 새롭운 등록 아이템 개수 / 100
Frist Search
소형 단진의 특별한 황금 주화 상자 새롭운 등록 아이템 개수 / 100
대형 단진의 특별한 황금 주화 상자 새롭운 등록 아이템 개수 / 1
소형 단진의 특별한 황금 주화 상자 새롭운 등록 아이템 개수 / 2
대형 단진의 특별한 황금 주화 상자 새롭운 등록 아이템 개수 / 2
소형 단진의 특별한 황금 주화 상자 새롭운 등록 아이템 개수 / 3
대형 단진의 특별한 황금 주화 상자 새롭운 등록 아이템 개수 / 1
소형 단진의 특별한 황금 주화 상자 새롭운 등록 아이템 개수 / 2
대형 단진의 특별한 황금 주화 상자 새롭운 등록 아이템 개수 / 1
소형 단진의 특별한 황금 주화 상자 새롭운 등록 아이템 개수 / 1
소형 단진의 특별한 황금 주화 상자 새롭운 등록 아이템 개수 / 2
대형 단진의 특별한 황금 주화 상자 새롭운 등록 아이템 개수 / 1
대형 단진의 특별한 황금 주화 상자 새롭운 등록 아이템 개수 / 3
대형 단진의 특별한 황금 주화 상자 새롭운 등록 아이템 개수 / 1
소형 단진의 특별한 황금 주화 상자 새롭운 등록 아이템 개수 / 1
소형 단진의 특별한 황금 주화 상자 새롭운 등록 아이템 개수 / 2
대형 단진의 특별한 황금 주화 상자 새롭운 등록 아이템 개수 / 1
소형 단진의 특별한 황금 주화 상자 새롭운 등록 아이템 개수 / 1
소형 단진의 특별한 황금 주화 상자 새롭운 등록 아이템 개수 / 1
소형 단진의 특별한 황금 주화 상자 새롭운 등록 아이템 개수 / 1
대형 단진의 특별한 황금 주화 상자 새롭운 등록 아이템 개수 / 1
소형 단진의 특별한 황금 주화 상자 새롭운 등록 아이템 개수 / 3
소형 단진의 특별한 황금 주화 상자 새롭운 등록 아이템 개수 / 1
소형 단진의 특별한 황금 주화 상자 새롭운 등록 아이템 개수 / 1
대형 단진의 특별한 황금 주화 상자 새롭운 등록 아이템 개수 / 2
대형 단진의 특별한 황금 주화 상자 

KeyboardInterrupt: 

In [6]:
import  mysql.connector

aws = mysql.connector.Connect(
    database = 'df',
    host = "dungeonandfighter-db.crnv1am9ifjd.ap-northeast-2.rds.amazonaws.com",
    port = 3306,
    user = 'sold_table',
    password = '1234'
)
cur = aws.cursor()

In [11]:
def excecuteDB(df):
    cols = df.columns
    insert_cols = ",".join(cols)
    for idx, row in df.iterrows():
        insert_value = ['"'+str(row[col])+'"' for col in cols]
        insert_value = ",".join(map(str,insert_value))
        query = 'INSERT INTO sold_history('+ insert_cols +') VALUES('+ insert_value +');'
        cur.execute(query)
    aws.commit()

In [8]:
# 아이템의 판매이력중 판매날짜 정보 가져오기
def get_auction_saled_record_from_db(target):
    cur = aws.cursor()
    query = f"Select soldDate from sold_history where itemId like '{target}'"
    cur.execute(query)
    df = pd.DataFrame(cur.fetchall(), columns={'soldDate'})
    
    return df['soldDate'].values


In [62]:
aws.close()

In [ ]:
# 테이블 생성 쿼리

# create_table = create_table[['soldDate', 'itemId' , 'itemName' , 'count' , 'unitPrice']]
# query = ''
# for col in create_table.columns:
#     varchar = len(str(create_table[col].iloc[0]))
#     if varchar < 15:
#         varchar = 16
#     elif varchar < 31:
#         varchar = 32
    
#     if col == 'itemName':
#         varchar = 256
        
#     query += col + " varchar(" + str(varchar) + "), "
    
# # print('create table sold_history (' + query[:-2] +');')
# cur.execute('create table sold_history (' + query[:-2] +');')